In [ ]:
import os
import math
import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision, torchvision.transforms as T


# Utilities
def count_params(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

def try_compute_flops(model, input_size=(1,3,32,32)):
    try:
        from thop import profile
        device = next(model.parameters()).device
        dummy = torch.randn(*input_size).to(device)
        model.eval()
        macs, params = profile(model, inputs=(dummy,), verbose=False)
        flops = 2 * macs
        return flops
    except Exception:
        return None


# Model building blocks
class DWConv(nn.Module):
    def __init__(self, dim, kernel=3):
        super().__init__()
        self.op = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=kernel, padding=kernel//2, groups=dim, bias=False),
            nn.BatchNorm2d(dim),
            nn.ReLU(inplace=True)
        )
    def forward(self, x, H, W):
        # x: [B, N, C] -> to [B, C, H, W]
        B, N, C = x.shape
        x2 = x.transpose(1,2).reshape(B, C, H, W)
        x2 = self.op(x2)
        x2 = x2.reshape(B, C, -1).transpose(1,2)
        return x2

class FFN(nn.Module):
    def __init__(self, dim, hidden_dim, drop=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(drop),
        )
    def forward(self, x):
        return self.net(x)

class CascadedGroupAttention(nn.Module):
    def __init__(self, dim, num_heads=4, qk_ratio=0.5, kv_expand_v=False):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"
        self.dim = dim
        self.h = num_heads
        self.head_dim = dim // num_heads


        self.qk_dim = max(1, int(self.head_dim * qk_ratio))


        self.q_projs = nn.ModuleList([nn.Linear(self.head_dim, self.qk_dim, bias=False) for _ in range(self.h)])
        self.k_projs = nn.ModuleList([nn.Linear(self.head_dim, self.qk_dim, bias=False) for _ in range(self.h)])
        self.v_projs = nn.ModuleList([nn.Linear(self.head_dim, self.head_dim, bias=False) for _ in range(self.h)])
        self.out_proj = nn.Linear(dim, dim)


        self.q_dw = nn.ModuleList([DWConv(self.qk_dim) for _ in range(self.h)])

    def forward(self, x, H, W):

        B, N, C = x.shape

        x_split = x.reshape(B, N, self.h, self.head_dim).permute(2,0,1,3)  

        head_outputs = []
        prev_out = None
        for j in range(self.h):
            xj = x_split[j]  
            if prev_out is not None:
                xj = xj + prev_out

           
            q = self.q_projs[j](xj)   # [B, N, qk_dim]
            k = self.k_projs[j](xj)   # [B, N, qk_dim]
            v = self.v_projs[j](xj)   # [B, N, head_dim]

            q = q.transpose(1,2).reshape(B*self.qk_dim, 1, H, W)  
            q = q.reshape(B, self.qk_dim, H, W).reshape(B, self.qk_dim, -1).transpose(1,2)[:,:N,:] 

            qk = torch.matmul(q, k.transpose(-2, -1))  # [B, N, N]
            qk = qk / math.sqrt(max(1.0, float(self.qk_dim)))
            attn = torch.softmax(qk, dim=-1)
            out_j = torch.matmul(attn, v)  # [B, N, head_dim]

            head_outputs.append(out_j)
            prev_out = out_j  

        out = torch.cat(head_outputs, dim=-1)  # [B, N, C]
        out = self.out_proj(out)
        return out

class SandwichBlock(nn.Module):
    def __init__(self, dim, num_heads=4, mlp_ratio=2.0, drop=0.0, qk_ratio=0.5):
        super().__init__()
        hidden = int(dim * mlp_ratio)
        self.ffn1 = FFN(dim, hidden, drop)
        self.dw1 = DWConv(dim)
        self.attn = CascadedGroupAttention(dim, num_heads=num_heads, qk_ratio=qk_ratio)
        self.dw2 = DWConv(dim)
        self.ffn2 = FFN(dim, hidden, drop)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.norm3 = nn.LayerNorm(dim)
        self.norm4 = nn.LayerNorm(dim)

    def forward(self, x, H, W):
        # x: [B, N, C]
        y = self.norm1(x)
        y = self.ffn1(y)
        x = x + y

        y = self.norm2(x)
        y = self.dw1(y, H, W)
        x = x + y

        y = self.norm3(x)
        y = self.attn(y, H, W)
        x = x + y

        y = self.norm4(x)
        y = self.ffn2(y)
        x = x + y
        return x

# -------------------------
# TinyEfficientViT 
# -------------------------
class TinyEfficientViT(nn.Module):
    def __init__(self, img_size=32, num_classes=100,
                 stage_channels=(64,128,192), stage_depths=(1,2,3),
                 stage_heads=(4,4,4), embed_dim=64, mlp_ratio=2.0, qk_ratio=0.5):
        super().__init__()
        self.img_size = img_size
        self.stem = nn.Sequential(
            nn.Conv2d(3, stage_channels[0], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(stage_channels[0]),
            nn.ReLU(inplace=True),
            nn.Conv2d(stage_channels[0], stage_channels[0], kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(stage_channels[0]),
            nn.ReLU(inplace=True),
        )
        self.stage_channels = stage_channels
        self.stage_depths = stage_depths
        self.stage_heads = stage_heads

        self.stage_blocks = nn.ModuleList()
        self.stage_proj = nn.ModuleList()

        for idx, (C, L, H) in enumerate(zip(stage_channels, stage_depths, stage_heads)):
            proj = nn.Sequential(
                nn.Conv2d(C, C, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(C),
                nn.ReLU(inplace=True),
            )
            self.stage_proj.append(proj)

            blocks = nn.ModuleList([SandwichBlock(dim=C, num_heads=H, mlp_ratio=mlp_ratio, qk_ratio=qk_ratio) for _ in range(L)])
            self.stage_blocks.append(blocks)

            if idx < len(stage_channels) - 1:
                down = nn.Conv2d(C, stage_channels[idx+1], kernel_size=3, stride=2, padding=1, bias=False)
                setattr(self, f"down_{idx}", nn.Sequential(down, nn.BatchNorm2d(stage_channels[idx+1]), nn.ReLU(inplace=True)))

        # classifier head
        final_C = stage_channels[-1]
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Linear(final_C, num_classes)

    def forward(self, x):
        B = x.shape[0]
        # stage 0 stem
        x = self.stem(x)  # [B, C1, H/2, W/2]
        # stage 0
        for blk in self.stage_blocks[0]:
            B, C, H, W = x.shape[0], x.shape[1], x.shape[2], x.shape[3]
            N = H * W
            tokens = x.reshape(B, C, -1).permute(0,2,1)  # [B, N, C]
            tokens = blk(tokens, H, W)
            x = tokens.permute(0,2,1).reshape(B, C, H, W)

        for idx in range(1, len(self.stage_blocks)):
            down = getattr(self, f"down_{idx-1}")
            x = down(x)  
            x = self.stage_proj[idx](x)
            # blocks
            for blk in self.stage_blocks[idx]:
                B, C, H, W = x.shape[0], x.shape[1], x.shape[2], x.shape[3]
                tokens = x.reshape(B, C, -1).permute(0,2,1)
                tokens = blk(tokens, H, W)
                x = tokens.permute(0,2,1).reshape(B, C, H, W)

        # head
        x = self.global_pool(x).reshape(B, -1)
        out = self.head(x)
        return out


# CIFAR-100 
def get_cifar100_loaders(bs=128, num_workers=4):
    t_train = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5071,0.4865,0.4409),(0.2673,0.2564,0.2761)),
    ])
    t_test = T.Compose([
        T.ToTensor(),
        T.Normalize((0.5071,0.4865,0.4409),(0.2673,0.2564,0.2761)),
    ])
    train = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=t_train)
    test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=t_test)
    return DataLoader(train, batch_size=bs, shuffle=True, num_workers=num_workers), \
           DataLoader(test, batch_size=bs, shuffle=False, num_workers=num_workers)

# Training loop, metrics, plotting
def train_one_epoch(model, loader, opt, device, loss_fn):
    model.train()
    total_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    pbar = tqdm(loader, leave=False)
    for x,y in pbar:
        x,y = x.to(device), y.to(device)
        opt.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()

        total_loss += loss.item() * x.size(0)
        total += x.size(0)
        pred = out.topk(5, dim=1)[1]
        correct_top1 += (pred[:,0] == y).sum().item()
        correct_top5 += sum([1 if y[i].item() in pred[i].tolist() else 0 for i in range(y.size(0))])

    avg_loss = total_loss / total
    top1 = correct_top1 / total
    top5 = correct_top5 / total
    return avg_loss, top1, top5

def evaluate(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            out = model(x)
            loss = loss_fn(out, y)
            total_loss += loss.item() * x.size(0)
            total += x.size(0)
            pred = out.topk(5, dim=1)[1]
            correct_top1 += (pred[:,0] == y).sum().item()
            correct_top5 += sum([1 if y[i].item() in pred[i].tolist() else 0 for i in range(y.size(0))])
    avg_loss = total_loss / total
    top1 = correct_top1 / total
    top5 = correct_top5 / total
    return avg_loss, top1, top5

def plot_metrics(history, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    epochs = len(history['train_loss'])
    # Loss
    plt.figure()
    plt.plot(range(1,epochs+1), history['train_loss'], label='train_loss')
    plt.plot(range(1,epochs+1), history['val_loss'], label='val_loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid()
    plt.title('Loss')
    plt.savefig(os.path.join(out_dir,'loss.png'), dpi=200)
    plt.close()
    # Accuracy
    plt.figure()
    plt.plot(range(1,epochs+1), np.array(history['train_top1'])*100, label='train_top1')
    plt.plot(range(1,epochs+1), np.array(history['val_top1'])*100, label='val_top1')
    plt.plot(range(1,epochs+1), np.array(history['val_top5'])*100, label='val_top5')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.grid()
    plt.title('Accuracy')
    plt.savefig(os.path.join(out_dir,'accuracy.png'), dpi=200)
    plt.close()


# Main experiment
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    batch_size = 128
    epochs = 150
    lr = 3e-4
    weight_decay = 0.05
    num_classes = 100

    stage_channels = (64, 128, 192)
    stage_depths = (1, 2, 3)
    stage_heads = (4, 4, 4)
    mlp_ratio = 2.0  
    qk_ratio = 0.25   

    print("Building model...")
    model = TinyEfficientViT(img_size=32, num_classes=num_classes,
                             stage_channels=stage_channels,
                             stage_depths=stage_depths,
                             stage_heads=stage_heads,
                             mlp_ratio=mlp_ratio,
                             qk_ratio=qk_ratio).to(device)

    total_params, trainable = count_params(model)
    print(f"Total params: {total_params:,} ({total_params/1e6:.3f}M) | Trainable: {trainable:,}")

    flops = try_compute_flops(model, input_size=(1,3,32,32))
    if flops is not None:
        print(f"FLOPs (approx): {flops:.2e}")
    else:
        print("thop not available — FLOPs not computed (install 'thop' to compute)")

    # Data
    train_loader, test_loader = get_cifar100_loaders(bs=batch_size, num_workers=4)

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    loss_fn = nn.CrossEntropyLoss()

    history = {'train_loss':[], 'train_top1':[], 'train_top5':[], 'val_loss':[], 'val_top1':[], 'val_top5':[]}
    best_acc = 0.0
    os.makedirs('outputs', exist_ok=True)

    for ep in range(1, epochs+1):
        t0 = time.time()
        train_loss, train_top1, train_top5 = train_one_epoch(model, train_loader, opt, device, loss_fn)
        val_loss, val_top1, val_top5 = evaluate(model, test_loader, device, loss_fn)
        sched.step()
        history['train_loss'].append(train_loss); history['train_top1'].append(train_top1); history['train_top5'].append(train_top5)
        history['val_loss'].append(val_loss); history['val_top1'].append(val_top1); history['val_top5'].append(val_top5)

        if val_top1 > best_acc:
            best_acc = val_top1
            torch.save({'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': opt.state_dict(),
                        'epoch': ep},
                       os.path.join('outputs', 'best_tinyefficientvit_cifar100.pth'))

        t1 = time.time()
        print(f"Epoch {ep:03d}/{epochs} | train_loss {train_loss:.4f} | train_top1 {train_top1*100:.2f}% | val_top1 {val_top1*100:.2f}% | val_top5 {val_top5*100:.2f}% | time {t1-t0:.1f}s")

        if ep % 25 == 0:
            torch.save({'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': opt.state_dict(),
                        'epoch': ep},
                       os.path.join('outputs', f'ckpt_ep{ep:03d}.pth'))

    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
                'epoch': epochs}, os.path.join('outputs', 'last_tinyefficientvit_cifar100.pth'))

    plot_metrics(history, out_dir='outputs')

    print("Training done. Best val top-1: {:.2f}%".format(best_acc*100))
    print("Saved outputs & plots to ./outputs")

if __name__ == '__main__':
    main()


Building model...
Total params: 1,829,828 (1.830M) | Trainable: 1,829,828
thop not available — FLOPs not computed (install 'thop' to compute)


100%|██████████| 169M/169M [00:05<00:00, 28.8MB/s]


Epoch 001/150 | train_loss 3.6706 | train_top1 13.58% | val_top1 21.74% | val_top5 49.66% | time 34.8s


Epoch 002/150 | train_loss 2.9327 | train_top1 26.03% | val_top1 30.11% | val_top5 61.88% | time 33.1s


Epoch 003/150 | train_loss 2.5629 | train_top1 33.27% | val_top1 36.88% | val_top5 70.04% | time 33.3s


...

Epoch 146/150 | train_loss 0.0043 | train_top1 99.93% | val_top1 66.07% | val_top5 87.95% | time 34.1s


Epoch 147/150 | train_loss 0.0040 | train_top1 99.95% | val_top1 66.19% | val_top5 87.94% | time 34.2s


Epoch 148/150 | train_loss 0.0044 | train_top1 99.92% | val_top1 66.10% | val_top5 88.03% | time 34.1s


Epoch 149/150 | train_loss 0.0042 | train_top1 99.93% | val_top1 66.24% | val_top5 87.99% | time 34.6s


Epoch 150/150 | train_loss 0.0043 | train_top1 99.91% | val_top1 66.04% | val_top5 88.04% | time 34.2s
Training done. Best val top-1: 66.24%
Saved outputs & plots to ./outputs
